In [1]:
import preprocessing
from sklearn.model_selection import KFold
import argparse
from model import *
from train import *
import torch.optim as optim


In [2]:
subjects_adj_before,subjects_labels_before = preprocessing.load_data(25629,25830)

/vol/biomedic3/bglocker/ugproj2324/nns20/y4/DGL/super-brains/GSR-Net/preprocessing.py:41: RuntimeWarning: Mean of empty slice
  col_mean = np.nanmean(roi, axis=0)


In [3]:
subjects_adj, subjects_labels, test_adj, test_labels = preprocessing.data()

In [4]:
# compare the adjacency matrix before and after preprocessing
if subjects_adj_before.all() == subjects_adj.all():
    print("The adjacency matrix before and after preprocessing is the same?")


The adjacency matrix before and after preprocessing is the same?


In [10]:
num_splt = 5
epochs = 200
lr = 0.0001
lmbda = 16
lr_dim = 160
hr_dim = 320
hidden_dim = 320
padding = 26

args = argparse.Namespace()
args.epochs = epochs
args.lr = lr
args.lmbda = lmbda
args.lr_dim = lr_dim
args.hr_dim = hr_dim
args.hidden_dim = hidden_dim
args.padding = padding


In [11]:
cv = KFold(n_splits=5, random_state=42, shuffle=True)

In [12]:
# ks = [0]
ks = [0.9, 0.7, 0.6, 0.5]
model = GSRNet(ks, args)

In [26]:
criterion = nn.MSELoss()

def train(model, optimizer, subjects_adj,subjects_labels, args):
  
  i = 0
  all_epochs_loss = []
  no_epochs = args.epochs

  for epoch in range(no_epochs):
      epoch_loss = []
      epoch_error = []

      for lr,hr in zip(subjects_adj,subjects_labels):

        model.train()
        optimizer.zero_grad()
        
        lr = torch.from_numpy(lr).type(torch.FloatTensor)
        hr = torch.from_numpy(hr).type(torch.FloatTensor)
        
        model_outputs,net_outs,start_gcn_outs,layer_outs = model(lr)
        model_outputs  = unpad(model_outputs, args.padding)

        padded_hr = pad_HR_adj(hr,args.padding)
        # eig_val_hr, U_hr = torch.symeig(padded_hr, eigenvectors=True,upper=True) # deprecated
        eig_val_hr, U_hr = torch.linalg.eigh(hr, UPLO='U')

        # print the shapes of the outputs
        # print(f"{net_outs.shape} ; {start_gcn_outs.shape}")
        # print(f"{model.layer.weights.shape} ; {U_hr.shape}")
        # print(f"{model_outputs.shape} ; {hr.shape}")
        
        # loss = args.lmbda * criterion(net_outs, start_gcn_outs) + criterion(model.layer.weights,U_hr) + criterion(model_outputs, hr) 
        loss = args.lmbda * criterion(net_outs, start_gcn_outs)+ criterion(model_outputs, hr) 
        
        error = criterion(model_outputs, hr)
        
        loss.backward()
        optimizer.step()

        epoch_loss.append(loss.item())
        epoch_error.append(error.item())
      
      i+=1
      print("Epoch: ",i, "Loss: ", np.mean(epoch_loss), "Error: ", np.mean(epoch_error)*100,"%")
      all_epochs_loss.append(np.mean(epoch_loss))

In [22]:
print(len(subjects_adj))
print(len(subjects_labels))

48
190


In [27]:
print(model)
optimizer = optim.Adam(model.parameters(), lr=args.lr)

for train_index, test_index in cv.split(subjects_adj):
    subjects_adj_train = subjects_adj[train_index]  # Get training data 
    subjects_adj_test = subjects_adj[test_index]   # Get testing data 
    subjects_ground_truth_train = subjects_labels[train_index]
    subjects_ground_truth_test = subjects_labels[test_index]

    train(model, optimizer, subjects_adj_train, subjects_ground_truth_train, args)
    test(model, subjects_adj_test, subjects_ground_truth_test, args)

GSRNet(
  (layer): GSRLayer()
  (net): GraphUnet(
    (start_gcn): GCN(
      (proj): Linear(in_features=160, out_features=320, bias=True)
      (drop): Dropout(p=0, inplace=False)
    )
    (bottom_gcn): GCN(
      (proj): Linear(in_features=320, out_features=320, bias=True)
      (drop): Dropout(p=0, inplace=False)
    )
    (end_gcn): GCN(
      (proj): Linear(in_features=640, out_features=320, bias=True)
      (drop): Dropout(p=0, inplace=False)
    )
  )
  (gc1): GraphConvolution()
  (gc2): GraphConvolution()
)
Epoch:  1 Loss:  0.0407714622683431 Error:  4.022855664554395 %
Epoch:  2 Loss:  0.037661418563833364 Error:  3.715254518350488 %
Epoch:  3 Loss:  0.03597334238063348 Error:  3.550805362235559 %
Epoch:  4 Loss:  0.03484366133220886 Error:  3.4377332923835833 %
Epoch:  5 Loss:  0.033975576393698394 Error:  3.3505304912595375 %
0.03379452973604202
0.038027070462703705
0.028532225638628006
0.034788038581609726
0.03315906971693039
0.03544401004910469
0.04250426962971687
0.03028